# Importing Uncleaned Data


In [2]:
import pandas as pd

# Load the dataset from the Final Datasets folder
file_path = 'Final Datasets/N1_Uncleaned_data.csv'
data = pd.read_csv(file_path)

# Display the first few rows and summary information
data_info = data.info()
data_head = data.head()

data_info, data_head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972174 entries, 0 to 972173
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   PUBLISH_DATE         972174 non-null  object 
 1   TRADING_NAME         972174 non-null  object 
 2   BRAND_DESCRIPTION    972174 non-null  object 
 3   PRODUCT_DESCRIPTION  972174 non-null  object 
 4   PRODUCT_PRICE        972174 non-null  float64
 5   ADDRESS              972174 non-null  object 
 6   LOCATION             972174 non-null  object 
 7   POSTCODE             972174 non-null  int64  
 8   AREA_DESCRIPTION     972174 non-null  object 
 9   REGION_DESCRIPTION   972174 non-null  object 
 10  Unnamed: 10          0 non-null       float64
 11  latitude             968780 non-null  float64
 12  longitude            968780 non-null  float64
dtypes: float64(4), int64(1), object(8)
memory usage: 96.4+ MB


(None,
   PUBLISH_DATE            TRADING_NAME BRAND_DESCRIPTION PRODUCT_DESCRIPTION  \
 0   01/01/2020          7-Eleven Ascot          7-Eleven                 ULP   
 1   01/01/2020       7-Eleven Balcatta          7-Eleven                 ULP   
 2   01/01/2020          7-Eleven Balga          7-Eleven                 ULP   
 3   01/01/2020  7-Eleven Banksia Grove          7-Eleven                 ULP   
 4   01/01/2020     7-Eleven Bassendean          7-Eleven                 ULP   
 
    PRODUCT_PRICE                 ADDRESS       LOCATION  POSTCODE  \
 0          156.5   194 Great Eastern Hwy          ASCOT      6104   
 1          153.9         174 Balcatta Rd       BALCATTA      6021   
 2          157.5         102 Princess Rd          BALGA      6061   
 3          157.5  1/300 Joseph Banks Bvd  BANKSIA GROVE      6031   
 4          157.5      302-318 Collier Rd     BASSENDEAN      6054   
 
   AREA_DESCRIPTION REGION_DESCRIPTION  Unnamed: 10   latitude   longitude  
 0   S

# Cleaning Data

In [3]:
#Remove the empty 'Unnamed: 10' column
data_cleaned = data.drop(columns=['Unnamed: 10'])

#Convert 'PUBLISH_DATE' to datetime format
data_cleaned['PUBLISH_DATE'] = pd.to_datetime(data_cleaned['PUBLISH_DATE'], format='%d/%m/%Y')

#Group by 'LOCATION' and calculate the mean latitude and longitude
location_means = data_cleaned.groupby('LOCATION')[['latitude', 'longitude']].mean()

#Impute missing values in 'latitude' and 'longitude' based on the 'LOCATION' means
data_cleaned['latitude'] = data_cleaned.apply(
    lambda row: location_means.loc[row['LOCATION'], 'latitude'] if pd.isna(row['latitude']) else row['latitude'], axis=1
)
data_cleaned['longitude'] = data_cleaned.apply(
    lambda row: location_means.loc[row['LOCATION'], 'longitude'] if pd.isna(row['longitude']) else row['longitude'], axis=1
)

# Check if there are any remaining missing values after imputation
missing_values_post_imputation = data_cleaned[['latitude', 'longitude']].isna().sum()

missing_values_post_imputation.head(), data_cleaned.head()

(latitude     0
 longitude    0
 dtype: int64,
   PUBLISH_DATE            TRADING_NAME BRAND_DESCRIPTION PRODUCT_DESCRIPTION  \
 0   2020-01-01          7-Eleven Ascot          7-Eleven                 ULP   
 1   2020-01-01       7-Eleven Balcatta          7-Eleven                 ULP   
 2   2020-01-01          7-Eleven Balga          7-Eleven                 ULP   
 3   2020-01-01  7-Eleven Banksia Grove          7-Eleven                 ULP   
 4   2020-01-01     7-Eleven Bassendean          7-Eleven                 ULP   
 
    PRODUCT_PRICE                 ADDRESS       LOCATION  POSTCODE  \
 0          156.5   194 Great Eastern Hwy          ASCOT      6104   
 1          153.9         174 Balcatta Rd       BALCATTA      6021   
 2          157.5         102 Princess Rd          BALGA      6061   
 3          157.5  1/300 Joseph Banks Bvd  BANKSIA GROVE      6031   
 4          157.5      302-318 Collier Rd     BASSENDEAN      6054   
 
   AREA_DESCRIPTION REGION_DESCRIPTION   la

## Outlier Detection 

In [5]:
# Outlier Detection in PRODUCT_PRICE using Interquartile Range (IQR)
Q1 = data_cleaned['PRODUCT_PRICE'].quantile(0.25)
Q3 = data_cleaned['PRODUCT_PRICE'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Flag the outliers
outliers = data_cleaned[(data_cleaned['PRODUCT_PRICE'] < lower_bound) | (data_cleaned['PRODUCT_PRICE'] > upper_bound)]
print(f"Number of outliers detected: {len(outliers)}")


Number of outliers detected: 0


#  Normalize text data


In [6]:
data_cleaned['TRADING_NAME'] = data_cleaned['TRADING_NAME'].str.lower().str.strip()
data_cleaned['BRAND_DESCRIPTION'] = data_cleaned['BRAND_DESCRIPTION'].str.lower().str.strip()
data_cleaned['REGION_DESCRIPTION'] = data_cleaned['REGION_DESCRIPTION'].str.lower().str.strip()

# Check for missing Values

In [7]:
missing_data = data_cleaned.isna().sum()
print("Missing data in each column:")
print(missing_data)

Missing data in each column:
PUBLISH_DATE           0
TRADING_NAME           0
BRAND_DESCRIPTION      0
PRODUCT_DESCRIPTION    0
PRODUCT_PRICE          0
ADDRESS                0
LOCATION               0
POSTCODE               0
AREA_DESCRIPTION       0
REGION_DESCRIPTION     0
latitude               0
longitude              0
dtype: int64


# Date Range Consistency

In [8]:
data_cleaned['PUBLISH_DATE'] = pd.to_datetime(data_cleaned['PUBLISH_DATE'], format='%d/%m/%Y', errors='coerce')


# Check the min and max dates

In [10]:
min_date = data_cleaned['PUBLISH_DATE'].min()
max_date = data_cleaned['PUBLISH_DATE'].max()
print(f"Date range: {min_date} to {max_date}")

Date range: 2020-01-01 00:00:00 to 2024-08-23 00:00:00


# Validate POSTCODE

In [11]:
# Ensure that postcodes are valid (4-digit numbers)
data_cleaned['POSTCODE'] = data_cleaned['POSTCODE'].astype(str).str.zfill(4)  # Ensure all postcodes have 4 digits
invalid_postcodes = data_cleaned[~data_cleaned['POSTCODE'].str.match(r'^\d{4}$')]

print(f"Number of invalid postcodes: {len(invalid_postcodes)}")


# Display the cleaned data and any detected issues
data_cleaned.head(), outliers.head(), invalid_postcodes.head()

Number of invalid postcodes: 0


(  PUBLISH_DATE            TRADING_NAME BRAND_DESCRIPTION PRODUCT_DESCRIPTION  \
 0   2020-01-01          7-eleven ascot          7-eleven                 ULP   
 1   2020-01-01       7-eleven balcatta          7-eleven                 ULP   
 2   2020-01-01          7-eleven balga          7-eleven                 ULP   
 3   2020-01-01  7-eleven banksia grove          7-eleven                 ULP   
 4   2020-01-01     7-eleven bassendean          7-eleven                 ULP   
 
    PRODUCT_PRICE                 ADDRESS       LOCATION POSTCODE  \
 0          156.5   194 Great Eastern Hwy          ASCOT     6104   
 1          153.9         174 Balcatta Rd       BALCATTA     6021   
 2          157.5         102 Princess Rd          BALGA     6061   
 3          157.5  1/300 Joseph Banks Bvd  BANKSIA GROVE     6031   
 4          157.5      302-318 Collier Rd     BASSENDEAN     6054   
 
   AREA_DESCRIPTION REGION_DESCRIPTION   latitude   longitude  
 0   South of River             

# Exporting Cleaned Data

In [13]:
# Export the cleaned data to a CSV file
output_file_path = 'Final Datasets/N1_Cleaned_fueldata.csv'
data_cleaned.to_csv(output_file_path, index=False)

print(f"Cleaned data saved to {output_file_path}")

Cleaned data saved to Final Datasets/N1_Cleaned_fueldata.csv
